# **Methylation Biomarkers for Predicting Cancer**

## **Deep Learning Approaches to Cancer Classification**

**Author:** Meg Hutch

**Date:** January 26, 2020

**Objective:** Use neural networks to classify colon, esophagus, liver, and stomach cancer patients and healthy subjects.

In [2]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors
import seaborn as sns

**Import Training, Testing, and Principal component data**

In [3]:
# Training set
mcTrain = pd.read_csv('C:\\Users\\User\\Box Sync/Projects/Multi_Cancer_DL/02_Processed_Data/mcTrain.csv')
# Testing set
mcTest = pd.read_csv('C:\\Users\\User\\Box Sync/Projects/Multi_Cancer_DL/02_Processed_Data/mcTest.csv')
# All Principal Components
principal_Df_ALL = pd.read_csv('C:\\Users\\User\\Box Sync/Projects/Multi_Cancer_DL/02_Processed_Data/principalDF_ALL.csv')
# Principal Components that make up 90% of the variance of the training set
genesTrain_transformed_90 = pd.read_csv('C:\\Users\\User\\Box Sync/Projects/Multi_Cancer_DL/02_Processed_Data/genesTrain_transformed_90.csv')
# Principal Components projected onto the test set
genesTest_transformed_90 = pd.read_csv('C:\\Users\\User\\Box Sync/Projects/Multi_Cancer_DL/02_Processed_Data/genesTest_transformed_90.csv')

**Pre-Process Data**

* Standarized all data: UPDATE: Not going to worry about this yet!
* Make sure that data is formatted correctly
* Structure the neural network architecture for multi-classifciation - (check loss function?)
* Determine how to do LOOCFV 
* The idea is that I will try and get high AUCs using the LOOCFV and then once I optimize that, I'll test on the testing set (is it cheating at all to test and then go back to change?

In [4]:
# remove genetic data from the mcTrain dataset
mcTrain = mcTrain[['seq_num','diagnosis', 'dilute_library_concentration', 'age', 'gender']]

# do the same for the testing set
mcTest = mcTest[['seq_num','diagnosis', 'dilute_library_concentration', 'age', 'gender']]

In [5]:
# rename the first column name of the PC dataframes
genesTrain_transformed_90.rename(columns={'Unnamed: 0':'seq_num'}, inplace=True)
genesTest_transformed_90.rename(columns={'Unnamed: 0':'seq_num'}, inplace=True)

In [6]:
# merge PCs with clinical/phenotypic data
mcTrain = pd.merge(mcTrain, genesTrain_transformed_90, how="left", on="seq_num") 
mcTest = pd.merge(mcTest, genesTest_transformed_90, how="left", on="seq_num") 

**Shuffle the training and test sets**

Currently, all disease states are in order - we don't want to feed to the network in order!

In [7]:
import random
random.seed(222020)
mcTrain = mcTrain.sample(frac=1, axis = 0).reset_index(drop=True) # frac = 1 returns all rows in random order
mcTest = mcTest.sample(frac=1, axis = 0).reset_index(drop=True)

**Create a new numeric index and drop seq_num**
For future code we want the index to be numeric

In [8]:
# Create new ids
mcTrain['id'] = mcTrain.index + 1
mcTest['id'] = mcTest.index + 239

# Drop num_seq
mcTrain= mcTrain.drop(columns=["seq_num"])
mcTest = mcTest.drop(columns=["seq_num"])

**Remove Labels (Diagnosis) from the datasets**

In [9]:
mcTrain_x = mcTrain.drop(columns=["diagnosis"])
mcTest_x = mcTest.drop(columns=["diagnosis"])

**Create Labeled Datasets**

In [10]:
mcTrain_y = mcTrain[['id','diagnosis']]
mcTest_y = mcTest[['id','diagnosis']]

In [11]:
# Examine the unique target variables
mcTrain_y.diagnosis.unique()

array(['ESCA', 'HEA', 'CRC', 'HCC', 'STAD'], dtype=object)

In [12]:
# Replace each outcome target with numerical value
mcTrain_y = mcTrain_y.replace('HEA', 0)
mcTrain_y = mcTrain_y.replace('CRC', 1)
mcTrain_y = mcTrain_y.replace('ESCA', 2)
mcTrain_y = mcTrain_y.replace('HCC', 3)
mcTrain_y = mcTrain_y.replace('STAD', 4)

mcTest_y = mcTest_y.replace('HEA', 0)
mcTest_y = mcTest_y.replace('CRC', 1)
mcTest_y = mcTest_y.replace('ESCA', 2)
mcTest_y = mcTest_y.replace('HCC', 3)
mcTest_y = mcTest_y.replace('STAD', 4)

**Convert seq_num id to index**

In [13]:
mcTrain_x = mcTrain_x.set_index('id')
mcTrain_y = mcTrain_y.set_index('id')

mcTest_x = mcTest_x.set_index('id')
mcTest_y = mcTest_y.set_index('id')

# Neural Netwo

In [14]:
# Import PyTorch packages
import torch
from torch import nn
from torchvision import datasets, transforms
from torch import optim
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import TensorDataset
import torch.nn.functional as F
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

In [ ]:
# define list for results
results_ls = []  

# Leave-one-out-cross-fold validation
for index in range (0,238):

    # X - features
    mcTrain_xy = mcTrain_x.drop(mcTrain_x.index[index])
    mcTrain_xy = np.array(mcTrain_xy, dtype = "float32")
    
    # y - target/outputs
    mcTrain_yz = mcTrain_y.drop(mcTrain_y.index[index]) 
    mcTrain_yz = np.array(mcTrain_yz, dtype = "float32")
    
    # reformatt into tensors
    xb = torch.from_numpy(mcTrain_xy)
    yb = torch.from_numpy(mcTrain_yz)
    
    # squeeze 
    yb = yb.squeeze(1) # function is used when we want to remove single-dimensional entries from the shape of an array.
    
    # add the equivalent training_test set
    mcTrain_test_x = mcTrain_x.iloc[[index]]
    mcTrain_test_x = np.array(mcTrain_test_x, dtype = "float32")
            
    # y - targets/outputs
    mcTrain_test_y = mcTrain_y.iloc[[index]]
    mcTrain_test_y = np.array(mcTrain_test_y, dtype = "float32")
        
    # Convert arrays into tensors
    test_xb = torch.from_numpy(mcTrain_test_x)
    test_yb = torch.from_numpy(mcTrain_test_y)
    
    # append all dfs
    #df_train_x.append(xb)
    #df_train_y.append(yb)
    
#### What I think I can do next is maybe forgo the df_train_x - list of dataframes, rather I may just be able to start iteratively running the code
    # Define the batchsize
    batch_size = 32

    # Combine the arrays
    trainloader = TensorDataset(xb, yb)
    
    # Training Loader
    trainloader = DataLoader(trainloader, batch_size, shuffle=True)
    
    # Build Model
    # Define the model with hidden layers - 50 inputs
    model = nn.Sequential(nn.Linear(55, 30),
                          nn.ReLU(),
                          nn.Linear(30, 5))
                      
                      
    # Set optimizer and learning rate
    #optimizer = optim.SGD(model.parameters(), lr=0.003)

    # Could also use Adam optimizer; similar to stochastic gradient descent, but uses momentum which can speed up the actual fitting process, and it also adjusts the learning rate for each of the individual parameters in the model
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # loss function
    criterion = nn.CrossEntropyLoss() #don't use with softmax or sigmoid- PyTorch manual indicates "This criterion combines nn.LogSoftmax() and nn.NLLLoss() in one single class."

    
    # Set epochs
    epochs = 100
    for e in range(epochs):
        running_loss = 0
        
        for xb, yb in trainloader:
            # Training pass
            output = model.forward(xb)
            loss = criterion(output, yb.long()) # Loss calculated from the output compared to the labels  
            # clear gradients
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
            running_loss += loss.item() # loss.item() gets the scalar value held in the loss. Running_loss = 0, 
            # += notation, says "Add a value and the variable and assigns the result to that variable." So, adds the running_loss (0) with loss.item and assigns to running_loss
        else:
            print("Epoch {}/{}, Training loss: {:.5f}".format(e+1, epochs, running_loss/len(trainloader)))

    #for index in range (0,3):
    # Apply the model to the whole testing dataset
    ps = model(test_xb)
        
    #print('Probabilities', ps[:10])

    # Obtain the top probability
    top_p, top_class = ps.topk(1, dim=1)
    #print('true vals', test_yb[:10])

    #print(ps, top_p, top_class) # ps: shows us the model predictions for each of the 5 classes
    # top_p: identifies the max of the classes
    # top_class: gives us the 0-4 classification
                
    # Drop the grad 
    top_p = top_p.detach().numpy()
    top_class = top_class.detach().numpy()

    # convert to integers
    top_class = top_class.astype(np.int)
    test_yb = test_yb.numpy()
    test_yb = test_yb.astype(np.int)
    #print('top class', top_class[:10])
    print('prediction:', top_class)
    print('true:', test_yb)
                
# compare top_class to test_yb
    #for index in range(0,3):
    if top_class == test_yb:                
        results = 1 # prediction and true value are equal
    else: 
        results = 0
    print('Results:', results)
    
    results_ls.append(results)
    #results_ls.append(results_ls)
    print(results_ls)  

Epoch 1/100, Training loss: 3.94995
Epoch 2/100, Training loss: 3.20284
Epoch 3/100, Training loss: 2.66662
Epoch 4/100, Training loss: 2.19775
Epoch 5/100, Training loss: 1.80500
Epoch 6/100, Training loss: 1.56068
Epoch 7/100, Training loss: 1.33554
Epoch 8/100, Training loss: 1.20272
Epoch 9/100, Training loss: 1.17092
Epoch 10/100, Training loss: 1.05937
Epoch 11/100, Training loss: 0.98517
Epoch 12/100, Training loss: 0.95149
Epoch 13/100, Training loss: 0.89809
Epoch 14/100, Training loss: 0.85579
Epoch 15/100, Training loss: 0.81415
Epoch 16/100, Training loss: 0.78545
Epoch 17/100, Training loss: 0.73365
Epoch 18/100, Training loss: 0.71497
Epoch 19/100, Training loss: 0.68724
Epoch 20/100, Training loss: 0.67746
Epoch 21/100, Training loss: 0.64506
Epoch 22/100, Training loss: 0.62835
Epoch 23/100, Training loss: 0.58372
Epoch 24/100, Training loss: 0.56063
Epoch 25/100, Training loss: 0.54184
Epoch 26/100, Training loss: 0.53071
Epoch 27/100, Training loss: 0.50940
Epoch 28/1

In [100]:
percent_correct = sum(results_ls)
percent_correct = percent_correct/len(mcTrain)*100
print('Percent Correct', percent_correct)

Percent Correct 54.621848739495796


**Updates Feb 2:** I have the code working in a way that runs...but I need to ensure that the code is running in the way that I want. For each training set (Training set - 1), we want to run the neural network and then test the prediction of the 1 sample left out. Then we want to aggregate the predictions (test error) in order to average and have a sense of model performance. As of now, when I increase the number of samples + epochs we are getting no testing errors which seems highly unlikely and makes me thinkt there is a problem with my for loop.

Most recent update: The append is working - should confirm that everything looks okay. Will also need to run all samples with more epochs. Additionally, I should try and create a function to test back which samples aren't being classified correctly. Will be interesting to look at. 

Update: I realzied that the gradients aren't clearing. This might be something with the for loop - but I think at the first for loop stage when we are creating the seprearte tests

Update: Got the code to work! Needed to specify the nerual network code within the for loop

Still need to figure out of if I should re-scale everything + test out different hyperparameters

**Monday Feb 3:**

* Standardize all data?
* Start running tests 
* Create plot with each of the running loss to see how many epochs until converge? Different color for each test?
* Determine which samples are being misclassified?
* Develop grid search for hyperparameter tuning?